In [1]:
from sklearn.svm import LinearSVC
from sklearn.datasets import load_iris
from sklearn.feature_selection import SelectFromModel
from sklearn.pipeline import Pipeline

import pandas, seaborn, numpy, collections, numpy, pickle
%matplotlib inline
from matplotlib import pyplot
import topiary
import mhctools
import mhcflurry
from varcode.common import memoize
pandas.set_option('display.max_columns', None)
import matplotlib


def print_full(x):
    pandas.set_option('display.max_rows', len(x))
    print(x)
    pandas.reset_option('display.max_rows')

matplotlib.rc("savefig", dpi=800)
#matplotlib.rc("savefig", dpi=72)
matplotlib.rc('text', usetex=False)
#reload(c)

%matplotlib inline
%config InlineBackend.figure_format = 'png'



/Users/tim/miniconda3/envs/py3k/lib/python3.5/site-packages/matplotlib/__init__.py:872: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))
Using Theano backend.


In [2]:
# Write out mutation counts
mutations = pandas.read_csv("../data/derived/annotated_mutations.with_mhc_binders.csv.bz2",
                            converters={'binding_peptides': lambda x: eval(x) if x else {}})
mutations

/Users/tim/miniconda3/envs/py3k/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,source_id,donor,genome,contig,interbase_start,interbase_end,ref,alt,sources,effect,gene,context_5_prime,context_3_prime,context_mutation,binding_affinity,binding_allele,dna_alt_reads,dna_ref_reads,dna_total_reads,rna_alt_reads,rna_ref_reads,rna_total_reads,binding_peptides,num_binders,indel,snv,mnv,confident,vaf,any_alt_reads,mutation_id,site_id,unique_to_treated
0,AOCS-001-1-7,AOCS-001,GRCh37,14,24774226,24774227,C,A,annotated_mutations,silent,NOP9,ACCTTCCTAAAGCGG,GAGAGGCTTGGGAAC,C>A,NaN,NaN,17.0,28.0,45.0,185.0,244.0,429.0,{},0.0,False,True,False,True,0.377777,True,14:24774226 C>A,14:24774226,False
1,AOCS-001-1-7,AOCS-001,GRCh37,X,19968971,19968972,T,C,annotated_mutations,p.I548M,CXorf23,GTCATTTGGATCTAT,ATTTTGATCAGAGTC,T>C,94.33,B*41:01,16.0,51.0,67.0,0.0,77.0,77.0,"{'IKMIDPNDLRH': 35670.23, 'KMIDPNDLRH': 2516.5...",5.0,False,True,False,True,0.238806,True,X:19968971 T>C,X:19968971,False
2,AOCS-001-1-7,AOCS-001,GRCh37,8,35406835,35406836,G,A,annotated_mutations,p.E44K,UNC5D,CTGATGGGATGGATT,GGGAAGGGCTTCGCC,C>T,80.84,A*11:01,14.0,39.0,53.0,0.0,0.0,0.0,"{'ALPKSIPSA': 16582.38, 'RGTDNGEALPK': 1624.13...",4.0,False,True,False,True,0.264150,True,8:35406835 G>A,8:35406835,False
3,AOCS-001-1-7,AOCS-001,GRCh37,7,47872830,47872831,A,G,annotated_mutations,p.L2065P,HUS1 PKD1L1,CTGCATCAGCCATTC,CTCTGGGAGTGGCAG,T>C,343.29,A*11:01,16.0,31.0,47.0,0.0,1.0,1.0,"{'RKQPASAIPS': 27527.16, 'SAIPSGSGRAQ': 22401....",2.0,False,True,False,True,0.340425,True,7:47872830 A>G,7:47872830,False
4,AOCS-001-1-7,AOCS-001,GRCh37,17,17721666,17721667,G,C,annotated_mutations,p.R394G,SREBF1,AAATCTGCTGTCTTG,GCAAGGCCATCGACT,C>G,43.42,C*15:02,12.0,22.0,34.0,200.0,54.0,254.0,"{'NKSAVLGKAI': 20376.71, 'SAVLGKAID': 39430.07...",2.0,False,True,False,True,0.352940,True,17:17721666 G>C,17:17721666,False
5,AOCS-001-1-7,AOCS-001,GRCh37,7,148169063,148169064,G,T,annotated_mutations,intergenic,None,CAAAGAAGGGCCACC,AATCTAAACCAGCAC,C>A,NaN,NaN,19.0,39.0,58.0,0.0,0.0,0.0,{},0.0,False,True,False,True,0.327586,True,7:148169063 G>T,7:148169063,False
6,AOCS-001-1-7,AOCS-001,GRCh37,1,107152783,107152784,G,A,annotated_mutations,intergenic,None,TCCACTGTGTTTGTT,ATCCCTTACAAAAGA,C>T,NaN,NaN,14.0,69.0,83.0,0.0,0.0,0.0,{},0.0,False,True,False,True,0.168674,True,1:107152783 G>A,1:107152783,False
7,AOCS-001-1-7,AOCS-001,GRCh37,2,142133268,142133269,T,A,annotated_mutations,intronic,LRP1B,GCATTAAATTCCCAA,ATTGCACTTTGGTCC,T>A,NaN,NaN,9.0,28.0,37.0,0.0,0.0,0.0,{},0.0,False,True,False,True,0.243243,True,2:142133268 T>A,2:142133268,False
8,AOCS-001-1-7,AOCS-001,GRCh37,2,143687576,143687577,G,A,annotated_mutations,intronic,KYNU,GCCCTACAAAGCCTT,TGTTGTGAGGGAAAA,C>T,NaN,NaN,11.0,55.0,66.0,0.0,0.0,0.0,{},0.0,False,True,False,True,0.166666,True,2:143687576 G>A,2:143687576,False
9,AOCS-001-1-7,AOCS-001,GRCh37,1,144017729,144017730,G,T,annotated_mutations,non-coding-transcript,SRGAP2B,TGTAAGTGCACCAAT,GACACTCTGTATCTA,C>A,NaN,NaN,26.0,158.0,184.0,0.0,0.0,0.0,{},0.0,False,True,False,True,0.141304,True,1:144017729 G>T,1:144017729,False


In [3]:
sources = pandas.read_csv("../data/sources.csv", index_col="source_id")
sources["source_id"] = sources.index
sources = sources.ix[sources.cohort == "AOCS"]
treated_paired_donors = set(d for d in sources.donor
                              if len(sources.ix[sources.donor == d].treated.unique()) > 1 )
treated_paired_donors
sources["treated_paired"] = sources.donor.isin(treated_paired_donors)
sources

,donor,cohort,library,treated,timepoint,metastasis,tissue_type,interval_days,assay,paired,source_id,treated_paired
source_id,,,,,,,,,,,,
AOCS-001-1/SP101515,AOCS-001,AOCS,WGS,False,primary,False,solid,0.0,WGS,False,AOCS-001-1/SP101515,False
AOCS-002-1/SP101517,AOCS-002,AOCS,WGS,False,primary,False,solid,0.0,WGS,False,AOCS-002-1/SP101517,False
AOCS-004-1/SP101519,AOCS-004,AOCS,WGS,False,primary,False,solid,0.0,WGS,False,AOCS-004-1/SP101519,False
AOCS-005-1/SP101521,AOCS-005,AOCS,WGS,False,primary,False,solid,0.0,WGS,False,AOCS-005-1/SP101521,False
AOCS-034-13/SP101524,AOCS-034,AOCS,WGS,True,recurrence,False,ascites,1597.0,WGS,True,AOCS-034-13/SP101524,True
AOCS-034-1/SP101523,AOCS-034,AOCS,WGS,False,primary,False,solid,0.0,WGS,True,AOCS-034-1/SP101523,True
AOCS-055-1/SP101526,AOCS-055,AOCS,WGS,False,primary,False,solid,0.0,WGS,False,AOCS-055-1/SP101526,False
AOCS-056-1/SP101528,AOCS-056,AOCS,WGS,False,primary,False,solid,0.0,WGS,False,AOCS-056-1/SP101528,False
AOCS-057-1/SP101530,AOCS-057,AOCS,WGS,False,primary,False,solid,0.0,WGS,False,AOCS-057-1/SP101530,False


In [4]:
muts_new = mutations.ix[[
    (sources.ix[s].treated) for s in mutations.source_id
]].ix[
    mutations.snv &
    mutations.donor.isin(treated_paired_donors) &
    mutations.unique_to_treated
].groupby(["source_id", "context_mutation_3p5p"]).interbase_start.count().fillna(0).unstack().fillna(0)
muts_new["kind"] = "new"
muts_new

KeyError: 'AOCS-001-1-7'

In [ ]:
muts_all = mutations.ix[
    mutations.snv
].groupby(["source_id", "context_mutation_3p5p"]).interbase_start.count().unstack().T.fillna(0)
#muts_all.columns = ["%s__all" % (x.replace("/", "_")) for x in muts_all.columns]
muts_all = muts_all.T
muts_all["kind"] = "all"

print(muts_all.isnull().sum().sum())
muts_all


In [ ]:
# STATEMENT_UNIQUE_TO_TREATED_COUNT
print(mutations.ix[[
    (sources.ix[s].treated) for s in mutations.source_id
]].ix[
    mutations.snv &
    mutations.donor.isin(treated_paired_donors) &
    mutations.unique_to_treated
].shape)

print(mutations.ix[[
    (sources.ix[s].treated) for s in mutations.source_id
]].ix[
    mutations.snv &
    mutations.donor.isin(treated_paired_donors)
].shape)

In [ ]:
# To be loaded into R to use deconstructSigs package
muts_r = pandas.concat([muts_all, muts_new])

muts_r.columns = [x.replace("(", "[").replace(")", "]") for x in muts_r.columns]
muts_r["source_id"] = muts_r.index
muts_r = muts_r[["source_id"] + list(muts_r.columns)[:-1]]
muts_r.to_csv("../data/mutation_contexts_counts.csv", index=False)
muts_r